In [3]:
# Load the model from the endpoint
response = requests.get(model_endpoint)

# Check if the request was successful
if response.status_code == 200:
    response_json = response.json()

    # Print the response to inspect its structure
    print(response_json)

    # Identify the correct key containing the model's state dictionary
    # Adjust this based on the structure of the JSON response
    model_state_dict = response_json.get("model_state_dict")

    if model_state_dict is not None:
        # Instantiate the model and load its state dictionary
        model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=2,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm="batch",
        ).to(device)

        # Load the model's state dictionary
        model.load_state_dict(model_state_dict)

        # Set the model to evaluation mode
        model.eval()

        # Perform inference using the model
        # (Assuming test_org_loader, sliding_window_inference, decollate_batch, and post_transforms are defined elsewhere)
        with torch.no_grad():
            for test_data in test_org_loader:
                test_inputs = test_data["image"].to(device)
                roi_size = (160, 160, 160)
                sw_batch_size = 4
                test_data["pred"] = sliding_window_inference(test_inputs, roi_size, sw_batch_size, model)

                test_data = [post_transforms(i) for i in decollate_batch(test_data)]
    else:
        print("No model_state_dict found in the response.")
else:
    print(f"Error: {response.status_code} - {response.reason}")


Error: 501 - Not Implemented
